In [1]:
import pandas as pd
import numpy as np
from ensemble_boxes import *
from tqdm import tqdm

In [2]:
# image_id : image id
# PredictionString : label, score, x(min), y(min), x(max), y(max)
sample=pd.read_csv('/opt/ml/myfolder/UniverseNet-master/submissions/universe101_fold0_5759.csv')
image_list=sample['image_id']

In [3]:
df_preds=[
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/DetectoRS.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/Nuree_swin_fold0.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/Nuree_swin_fold1.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/Nuree_swin_fold2.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/Nuree_swin_fold3.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/Nuree_swin_fold4.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/swin_s_basic_fold0.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/swins_6013_all_data.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/swint_allcustomdata_5712.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/SwinT_epoch50_resize512_LB5543.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/swint_kfold_5732.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_fold2_5559.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_swa_fold0_5769.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_swa_fold1_5726.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_swa_fold3_5727.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_swa_fold4_5767.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe101_swa_kfold_0.6102.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/universe50_5472.csv"),
    pd.read_csv("/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/vfnetx_alldata_swa_5558.csv"),
]
model_count = len(df_preds)

In [4]:
def get_model_pred(df_pred):
    pred_list = df_pred['PredictionString'].tolist()
    result=[]
    for image_pred in pred_list:
        pred_line=image_pred.split(' ')[:-1]
        pred_line=np.array(pred_line).reshape(-1,6).transpose()
        boxes_count=pred_line.shape[1]
        labels=pred_line[0].astype(np.int32)
        scores=pred_line[1].astype(np.float32)
        boxes=[]
        for idx in range(boxes_count):
            pred_boxes=[]
            for loc in range(2,6):
                pred_boxes.append(float(pred_line[loc][idx]))
            boxes.append(pred_boxes)

        boxes = np.array(boxes)/512.

        result.append({
                    'boxes': boxes,
                    'scores': scores,
                    'labels': labels,})
    return result

In [5]:
model_prediction=[]
for df_pred in df_preds:
    model_prediction.append(get_model_pred(df_pred))

In [6]:
def run_wbf(prediction, model_count, image_index, iou_thr=0.5, skip_box_thr=0.05, weights=None):
    boxes = [prediction[model_idx][image_index]['boxes'].tolist() for model_idx in range(0,model_count)]
    scores = [prediction[model_idx][image_index]['scores'].tolist() for model_idx in range(0,model_count)]
    labels = [prediction[model_idx][image_index]['labels'].tolist() for model_idx in range(0,model_count)]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*512.
    return boxes, scores, labels

In [7]:
prediction_strings = []
file_names = []
for index, image_id in enumerate(tqdm(image_list)):
    boxes, scores, labels = run_wbf(model_prediction,model_count,index, iou_thr=0.6, skip_box_thr=0.125, weights=[0.35, 0.35, 0.3])
    
    prediction_string = ''
    for box, score, label in zip(boxes, scores, labels):
        prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
            box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

  1%|          | 9/837 [00:01<02:46,  4.96it/s]/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
  6%|▌         | 47/837 [00:02<00:33, 23.61it/s]/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
100%|██████████| 837/837 [01:00<00:00, 13.83it/s]


In [8]:
submission = pd.DataFrame()
submission['image_id'] = file_names
submission['PredictionString'] = prediction_strings
submission.to_csv(f'/opt/ml/myfolder/UniverseNet-master/work_dirs/all_csv/my_Ensemble.csv', index=None)
print(submission.head())

               image_id                                   PredictionString
0  batch_01_vt/0021.jpg  8 0.9937610030174255 173.1389617919922 234.696...
1  batch_01_vt/0028.jpg  8 0.9615394473075867 158.9316864013672 56.0507...
2  batch_01_vt/0031.jpg  1 0.9413436055183411 71.09355163574219 168.804...
3  batch_01_vt/0032.jpg  8 0.9201588034629822 43.006370544433594 124.55...
4  batch_01_vt/0070.jpg  8 0.9809255003929138 159.8540496826172 348.970...
